In [ ]:
from arango import ArangoClient
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta, timezone
import pyodbc
import struct

In [ ]:
arango_server = os.environ["ARANGO_HOST"]
arango_db = os.environ["ARANGO_DATABASE"]
arango_password = os.environ["ARANGO_PASSWORD"]
arango_user = os.environ["ARANGO_USER"]

In [ ]:
client = ArangoClient(hosts=arango_server)
db = client.db(arango_db, username=arango_user, password=arango_password)
cursor = db.aql.execute(
    r"LET DPTime = ( for doc in transactions return doc.report.metaData.prcgTmDP ) LET CRSPTime = (for doc in transactions return doc.report.metaData.prcgTmCRSP) LET RuleTime = (for doc in transactions FOR tp IN doc.report.tadpResult.channelResult[0].typologyResult FOR rule IN tp.ruleResults COLLECT prcgTm = rule.prcgTm RETURN  prcgTm ) LET TPTime = (for doc in transactions FOR tp IN doc.report.tadpResult.channelResult[0].typologyResult COLLECT prcgTm = tp.prcgTm RETURN  prcgTm ) LET CADPTime = (for doc in transactions return doc.report.tadpResult.channelResult[0].prcgTm) LET TADPTime = (for doc in transactions return   doc.report.tadpResult.prcgTm) LET E2ETimes = ( for doc in transactions COLLECT AGGREGATE mn= MIN(doc.report.timestamp), mx= MAX(doc.report.timestamp)  return {startTime: mn, endTime: mx, amount: LENGTH(transactions), elapsed: DATE_DIFF(mn, mx, 's'), ftps: LENGTH(transactions)/DATE_DIFF(mn, mx, 's')} ) LET DPStats = ( for doc in DPTime COLLECT AGGREGATE mn=MIN(doc), mx= MAX(doc), ave= AVERAGE(doc) return {mn:mn/1000000, mx:mx/1000000, ave:ave/1000000, mea:MEDIAN(DPTime)/1000000,ninety:PERCENTILE(DPTime, 90)/1000000,ninetynine:PERCENTILE(DPTime, 99)/1000000 } ) LET CRSPStats = ( for doc in CRSPTime COLLECT AGGREGATE mn= MIN(doc), mx= MAX(doc), ave= AVERAGE(doc)  return {mn:mn/1000000, mx:mx/1000000, ave:ave/1000000, mea:MEDIAN(CRSPTime)/1000000,ninety:PERCENTILE(CRSPTime, 90)/1000000,ninetynine:PERCENTILE(CRSPTime, 99)/1000000} ) LET RuleStats = ( for doc in RuleTime COLLECT AGGREGATE mn= MIN(doc), mx= MAX(doc), ave= AVERAGE(doc)  return {mn:mn/1000000, mx:mx/1000000, ave:ave/1000000, mea:MEDIAN(TPTime)/1000000,ninety:PERCENTILE(TPTime, 90)/1000000,ninetynine:PERCENTILE(TPTime, 99)/1000000} ) LET TPStats = ( for doc in TPTime COLLECT AGGREGATE mn= MIN(doc), mx= MAX(doc), ave= AVERAGE(doc)  return {mn:mn/1000000, mx:mx/1000000, ave:ave/1000000, mea:MEDIAN(TPTime)/1000000,ninety:PERCENTILE(TPTime, 90)/1000000,ninetynine:PERCENTILE(TPTime, 99)/1000000} ) LET CADPStats = ( for doc in CADPTime COLLECT AGGREGATE mn= MIN(doc), mx= MAX(doc), ave= AVERAGE(doc)  return {mn:mn/1000000, mx:mx/1000000, ave:ave/1000000, mea:MEDIAN(CADPTime)/1000000,ninety:PERCENTILE(CADPTime, 90)/1000000,ninetynine:PERCENTILE(CADPTime, 99)/1000000} ) LET TADPStats = ( for doc in TADPTime COLLECT AGGREGATE mn= MIN(doc), mx= MAX(doc), ave= AVERAGE(doc)  return {mn:mn/1000000, mx:mx/1000000, ave:ave/1000000, mea:MEDIAN(TADPTime)/1000000,ninety:PERCENTILE(TADPTime, 90)/1000000,ninetynine:PERCENTILE(TADPTime, 99)/1000000} ) RETURN {     startTime:E2ETimes[0].startTime,     endTime:E2ETimes[0].endTime,     amount:E2ETimes[0].amount,     elapsed:E2ETimes[0].elapsed,     ftps:E2ETimes[0].ftps,     DPStats,     CRSPStats,     RuleStats,     TPStats,     CADPStats,     TADPStats, DPTime, CRSPTime, RuleTime, TPTime, CADPTime, TADPTime }"
)
documents = [document for document in cursor]


run_details = documents[0]
cursor = db.aql.execute(
    r"LET typologiesArr = ( FOR doc IN transactions FOR typologies IN doc.report.tadpResult.channelResult[0].typologyResult[*] RETURN typologies ) LET rulesArr = ( FOR typologies IN typologiesArr FOR rules IN typologies.ruleResults[*] RETURN rules ) LET ruleTimesById = ( FOR rules IN rulesArr COLLECT ruleId = rules.id INTO rules RETURN {rule: ruleId, prcgTm: {mn: MIN(rules[*].rules.prcgTm), mx: MAX(rules[*].rules.prcgTm), ave: AVERAGE(rules[*].rules.prcgTm), ninety: PERCENTILE(rules[*].rules.prcgTm,90), ninetyfive: PERCENTILE(rules[*].rules.prcgTm,95), ninetynine: PERCENTILE(rules[*].rules.prcgTm,99)}} ) LET RuleStatsMs = ( FOR obj in ruleTimesById RETURN {ruleId: obj.rule, mn: obj.prcgTm.mn/1000000, mx: obj.prcgTm.mx/1000000, ave: obj.prcgTm.ave/1000000, ninety: obj.prcgTm.ninety/1000000,ninetyfive: obj.prcgTm.ninetyfive/1000000,ninetynine: obj.prcgTm.ninetynine/1000000} ) RETURN RuleStatsMs"
)
rules = documents = [document for document in cursor]

processors = {
    key: value[0] for (key, value) in run_details.items() if type(value) == list
}


In [ ]:
start_time = run_details["startTime"]
end_time = run_details["endTime"]
dp_times = run_details["DPTime"]
crsp_times = run_details["CRSPTime"]
rules_times = run_details["RuleTime"]
tp_times = run_details["TPTime"]
cadp_times = run_details["CADPTime"]
tadp_times = run_details["TADPTime"]

# Convert dataset to dataframe

In [ ]:
df = pd.DataFrame.from_dict(processors)
df2 = pd.DataFrame(rules[0])
df = df.drop(columns=["DPTime","CRSPTime","RuleTime","TPTime","CADPTime","TADPTime"])

df2 = df2.set_index("ruleId")

#df_expanded = df2["prcgTm"].apply(pd.Series)

#update = df.drop("mx")

df_no_max = df2.drop(columns=["mx"])

### Plot Rule Performance

In [ ]:
df.plot(kind="bar")


plt.tight_layout()
plt.ylabel("duration")
plt.title("Run summary")
plt.gca().xaxis.set_tick_params(rotation=0)

df_no_max.plot(kind="bar")
plt.tight_layout()
plt.ylabel("duration")
plt.title("Rule Performance")

In [ ]:
def histogram(data, processor):
    plt.hist(data, bins=50, edgecolor="black",alpha=0.6)
    plt.title(f"Frequency Distribution {processor}")
    plt.xlabel("duration")
    plt.ylabel("count")
    mean = np.mean(data)
    total = sum(data)
    plt.axvline(mean, linestyle="dashed",color="red",label="average")
    plt.legend()
    plt.tight_layout()
    plt.show()
    return mean

dp_mean = histogram(dp_times, "DP")
crsp_mean = histogram(crsp_times, "CRSP")
rules_mean = histogram(rules_times, "Rules")
tp_mean = histogram(tp_times, "TP")
cadp_mean = histogram(cadp_times, "CADP")
tadp_mean = histogram(tadp_times, "TADP")

In [ ]:
plt.pie([rules_mean, tp_mean, cadp_mean, tadp_mean, crsp_mean],labels=["Rules", "TP", "CADP", "TADP", "CRSP"])
plt.tight_layout()

In [ ]:
start_time_dt = datetime.fromisoformat(start_time)
end_time_dt = datetime.fromisoformat(end_time)
time_difference = end_time_dt.timestamp() - start_time_dt.timestamp()

In [ ]:
server = os.environ["SQL_HOST"]
database = os.environ["SQL_DB"]
username = os.environ["SQL_USER"]
password = os.environ["SQL_PASS"]
driver = os.environ["SQL_DRIVER"]
count = int(os.environ["PREVIOUS_BEST_COUNT"])

In [ ]:
insert_stmnt = """
INSERT dbo.Run (
crsp_avg,
crsp_min,
crsp_max,
rules_avg,
rules_min,
rules_max,
tadp_min,
tadp_avg,
tadp_max,
cadp_min,
cadp_max,
cad_avg,
dp_min,
dp_avg,
dp_max,
transaction_count,
start_time,
end_time,
avg_duration
) OUTPUT INSERTED.id 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""

select_stmnt = f"SELECT TOP {count} * FROM dbo.Run ORDER BY avg_duration;"

previous_runs = {}

In [ ]:
def handle_datetimeoffset(dto_value):
    # ref: https://github.com/mkleehammer/pyodbc/issues/134#issuecomment-281739794
    tup = struct.unpack("<6hI2h", dto_value)  # e.g., (2017, 3, 16, 10, 35, 18, 500000000, -6, 0)
    return datetime(tup[0], tup[1], tup[2], tup[3], tup[4], tup[5], tup[6] // 1000,
                    timezone(timedelta(hours=tup[7], minutes=tup[8])))

In [ ]:
with pyodbc.connect('DRIVER='+driver+';SERVER=tcp:'+server+';DATABASE='+database+';UID='+username+';PWD='+ password) as conn:
    document_count = [document for document in db.aql.execute("RETURN LENGTH(transactions)")]

    with conn.cursor() as cursor:
        conn.add_output_converter(-155, handle_datetimeoffset)
        cursor.execute(select_stmnt)
        previous_runs = pd.DataFrame.from_records(cursor.fetchall(), columns=[col[0] for col in cursor.description])
        
        cursor.execute(
            insert_stmnt,
            np.mean(crsp_times),
            min(crsp_times),
            max(crsp_times),
            np.mean(rules_times),
            min(rules_times),
            max(rules_times),
            min(tadp_times),
            np.mean(tadp_times),
            max(tadp_times),
            min(cadp_times),
            max(cadp_times),
            np.mean(cadp_times),
            min(dp_times),
            np.mean(dp_times),
            max(dp_times),
            document_count[0],
            start_time_dt,
            end_time_dt,
            time_difference / document_count[0]
        )
        resultId = cursor.fetchval()
        print(f"Run ID : {resultId} inserted")
        conn.commit()

In [ ]:
def vis_comparison(columns, previous_runs, current, processor):
    columns.append("id")
    current_vis = previous_runs[columns]
    current_vis = current_vis.set_index("id")
    current_vis = current_vis.astype(float)
    df2 = {'current_avg': np.mean(current), 'current_min': min(current), 'current_max': max(current) }
    current_vis = pd.concat([current_vis, pd.DataFrame([df2])], ignore_index=True)
    current_vis.plot(kind="bar")
    plt.tight_layout()
    plt.title(f"{processor} vs previous best")

In [ ]:
vis_comparison(["crsp_avg", "crsp_min", "crsp_max"], previous_runs, crsp_times, "CRSP")
vis_comparison(["rules_avg", "rules_min", "rules_max"], previous_runs, rules_times, "Rules")
vis_comparison(["tadp_avg", "tadp_min", "tadp_max"], previous_runs, tadp_times, "TADP")
vis_comparison(["cad_avg", "cadp_min", "cadp_max"], previous_runs, cadp_times, "CADP")
vis_comparison(["dp_avg", "dp_min", "dp_max"], previous_runs, dp_times, "DP")